In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR
import matplotlib.pyplot as plt
import scipy as sp

In [ ]:
edges, number, fobs, exname = ss.example5(print_test=False)
hc_bg, hc_ss, ssidx, hsamp, bgpar, sspar = ss.ss_by_cdefs(edges, number, 30, params=True)
# example 5
dur5 = 10.0*YR
cad5 = .2*YR
# dur4 = 5.0*YR/3.1557600
# cad4 = .2*YR/3.1557600

# Rosado et al 2015

## For GWB
PDF in the absence of a GWB
$$ p_0(S) = \frac{1}{\sqrt{2\pi\sigma_0^2} }e^{-\frac{(S-\mu_0)^2}{2\sigma_0^2}} $$
PDF if GWB is present in the data
 $$ p_1(S) = \frac{1}{\sqrt{2\pi\sigma_1^2} }e^{-\frac{(S-\mu_1)^2}{2\sigma_1^2}} $$

cross correlation
$$ S = \int_{-T/2}^{T/2} dt \int_{-T/2}^{T/2} dt' s_i(t) s_j(t') Q(t,t')$$
$T$ = observing time, $s_i(t)$ and $s_j(t)$ are different pulsar data, $Q(t,t')$ = filter function, chosen to maximize DP for a fixed FAP $=\alpha_0=0.001$ (Neyman-Pearson criterion)


## For single sources
PDF of $\mathcal{F}_e$ - statistic in absence of signal
$$ p_0(\mathcal{F}_e) = \mathcal{F}_e e^{-\mathcal{F}_e}$$
PDF of $\mathcal{F}_e$ - statistic if signal is present
$$ p_1(\mathcal{F}_e, \rho) = \frac{(2\mathcal{F}_e)^{1/2}}{\rho} I_1 (\rho \sqrt{2 \mathcal{F}_e}) e^{-\mathcal{F}_e - 1/2\rho^2} $$
where $I_1(x)$ = the modified Bessel function of the first kind of order 1 and
 $\rho$ = the optimal $S/N_S = \big[ \sum_{i=1}^M S/N_i^2 \big]^{1/2}$

## Detection probability
for single sources: $ \gamma_S(t) = \int_0^t p_s(t') dt'$ \
for background: $ \gamma_B(t) = \int_0^t p_B(t') dt'$

## Variables
* $\alpha$ = false alarm probability
* $\gamma$ = detection probability
* $S_T$ = threshold signal

# SNR
A-statistic: 
$$ S/N_A = \mu_1/\sigma_0$$
B-statistic:
$$ S/N_B = \mu_1 / \sigma_1 $$

In [ ]:
def SNR_A(mu_1, sigma_0):
    """ Calculate the SNR for the A-statistic S/N_A 
    
    Parameters
    ----------
    mu_1 : scalar
        Mean of GWB PDF.
    sigma_0 : scalar
        Standard deviation of noise processes.

    Returns
    -------
    SNR_A : scalar
        Signal to noise ratio for the A-statistic.
    """
    return mu_1/sigma_0

def SNR_B(mu_1, sigma_1):
    """ Calculate the SNR for the B-statistic S/N_B.

    Parameters
    ----------
    mu_1 : scalar
        Mean of GWB PDF.
    sigma_1 : scalar
        Standard deviation of the GWB PDf.

    Returns
    -------
    SNR_B : scalar
        Signal to noise ratio for the B-statistic.
    """
    return mu_1/sigma_1

Threshold signal to noise to have FAP $\alpha <\alpha_0$ and detection probability $\gamma > \gamma_0$

$$ \mathrm{S/N^T_A} = \sqrt{2} \big[ \mathrm{erfc}^{-1}(2\alpha_0) - \frac{\sigma_1}{\sigma_0}\mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

$$\mathrm{S/N^T_B} = \sqrt{2} \big[ \frac{\sigma_0}{\sigma_1} \mathrm{erfc}^{-1}(2\alpha_0) - \mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

$$ \mathrm{S/N^T} \approx \sqrt{2} \big[ \mathrm{erfc}^{-1}(2\alpha_0) - \mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

In [ ]:
def SNR_A_thresh(sigma_0, sigma_1, alpha_0, gamma_0):
    """ Calculate the threshold SNR for the A-statistic S/N^T_A 
    to have a FAP < alpha_0 and DP > gamma_0.
    
    Parameters
    ----------
    sigma_0 : scalar
        Standard deviation of noise processes.
    sigma_1 : scalar
        Standard deviation of the GWB
    alpha_0 : scalar
        False alarm probability max.
    gamma_0 : scalar
        Detection probability min.

    Returns
    -------
    SNT_A : scalar
        Signal to noise ratio for the A-statistic.

    Follows Rosado et al. 2015 Eq. (18)
    """
    SNT_A = np.sqrt(2) * (sp.erfcinv(2*alpha_0) 
                          - sigma_1/sigma_0 * sp.erfcinv(2*gamma_0))
    return SNT_A

def SNR_B_thresh(sigma_0, sigma_1, alpha_0, gamma_0):
    """ Calculate the threshold SNR for the B-statistic S/N^T_B
    to have a FAP < alpha_0 and DP > gamma_0.
    
    Parameters
    ----------
    sigma_0 : scalar
        Standard deviation of noise processes.
    sigma_1 : scalar
        Standard deviation of the GWB
    alpha_0 : scalar
        False alarm probability max.
    gamma_0 : scalar
        Detection probability min.

    Returns
    -------
    SNT_B : scalar
        Signal to noise ratio for the A-statistic.

    Follows Rosado et al. 2015 Eq. (19)
    """
    SNT_B = np.sqrt(2) * (sigma_0/sigma_1 * sp.erfcinv(2*alpha_0) 
                          - sp.erfcinv(2*gamma_0))
    return SNT_B

def SNR_approx_thresh(alpha_0, gamma_0):
    """ Calculate the approximate threshold SNR S/N^T for FAP < alpha_0 and DP > gamma_0/
    This approximates sigma_0 ~ sigma_1
    
    Parameters
    ----------
    alpha_0 : scalar
        False alarm probability maximum.
    gamma_0 : scalar
        Detection probability minimum.

    Returns
    -------
    SNT : scalar
        Signal to noise ratio threshold, approximated.

    Follows Rosado et al. 2015 Eq. (17)
    """
    SNT = np.sqrt(2) * (sp.erfcinv(2*alpha_0) - sp.erfcinv(2*gamma_0))
    return SNT

# Overlap Reduction Function
$$ \Gamma_{ij} = \frac{3}{2} \gamma_{ij} \ln (\gamma_{ij}) - \frac{1}{4} \gamma_{ij} + \frac{1}{2} + \frac{1}{2}\delta_{ij} $$
$$ \gamma_{ij} = [1-\cos (\theta_{ij})]/2$$

In [ ]:
def gammaij_from_thetaij(theta_ij):
    """ Calcualte gamma_ij for two pulsars of relative angle theta_ij.
    
    Parameters
    ----------
    theta_ij : scalar 
        Relative angular position between the ith and jth pulsars.

    Returns
    -------
    gamma_ij : scalar 
        [1 - cos(theta_ij)]/2

    """
    return (1-np.cos(theta_ij))/2

# TODO: Make this a function of theta AND phi
def thetaij_from_thetai_thetaj(theta_i, theta_j):
    """ Calcualte relative angle between two pulsars with angular positions theta_i and theta_j.
    
    Parameters
    ----------
    theta_i : scalar 
        Angular position in the sky of the ith pulsar.
    theta_j : scalar 
        Angular position in the sky of the jth pulsar.

    Returns
    -------
    theta_ij : scalar 
        Relative angular position between the ith and jth pulsar.

    """
    return np.abs(theta_i - theta_j)

def dirac_delta(i,j):
    """ Calculate the dirac delta function of i,j.
    Parameters
    ----------
    i : int
    j : int

    Returns
    ------- 
    dirac_ij : int
        Dirac delta function of i and j

    """
    if(i==j): return 1
    else: return 0

def overlap_from_gammaij_diracij(gamma_ij, dirac_ij):
    """ Calculate Gamma_i,j as a function of gamma_i,j and diracdelta_i,j

    Parameters
    ----------
    gamma_ij : scalar 
        (1-cos(theta_ij))/2
    dirac_ij : scalar 
        Dirac delta function of i and j.

    Returns
    -------
    Gamma_ij : scalar 
        Overlap reduction function of the ith and jth pulsars.

    Follows Rosado et al. 2015 Eq. (24)
    """
    
    Gamma_ij = (3/2 * gamma_ij *np.log(gamma_ij)
            - 1/4 * gamma_ij
            + 1/2 + dirac_ij)
    return Gamma_ij

def overlap_reduction_function(theta_i, theta_j, i, j):
    """ Calculate the overlap reduction function Gamma_i,j as a function of theta_i, theta_j, i, and j.
    
    Parameters
    ----------
    theta_i : scalar
        Angular position of the ith pulsar.
    theta_j : scalar
        Angular position of the jth pulsar.
    i : int
        index of the ith pulsar
    j : int
        index of the jth pulsar

    Returns
    -------
    Gamma_ij : scalar
        The overlap reduction function of the ith and jth pulsars.
    """
    dirac_ij = dirac_delta(i, j)
    theta_ij = thetaij_from_thetai_thetaj(theta_i, theta_j)
    gamma_ij = gammaij_from_thetaij(theta_ij)
    # print('dirac_ij:', dirac_ij, '\ntheta_ij', theta_ij, '\ngamma_ij', gamma_ij)
    Gamma_ij = overlap_from_gammaij_diracij(gamma_ij, dirac_ij)
    if(np.isnan(Gamma_ij) and i!=j):
        print('Gamma_%d,%d is nan, set to 0' % (i,j))
        return 0
    return Gamma_ij

In [ ]:
THETAS = np.array([0, np.pi/6, np.pi, 7/6*np.pi]) # (P,) 1Darray of scalars, angular sky position of each pulsar
print(THETAS)

num = len(THETAS) # number of pulsars, P
Gamma_ij = np.zeros((num, num)) # (P,P) 2Darray of scalars, Overlap reduction function between all puolsar
for ii in range(num):
    for jj in range(num):
        Gamma_ij[ii,jj] = overlap_reduction_function(THETAS[ii], THETAS[jj], ii, jj)
print(Gamma_ij)

### Plot Gamma function

Larger $\Gamma_{ij}$ means we expect more overlap right, so what does negative $\Gamma_{ij}$ mean?

Largest overlap at $\theta_i \sim \theta_j$, next largest at $\theta_i \sim \theta_j + \pi$ e.g. $\theta_{ij} \sim 180\degree$, smallest at $\theta_{ij} \sim 90\degree$ perpendicular in the sky

In [ ]:
theta_arr = np.linspace(0,2*np.pi,100)
gamma_arr = (1-np.cos(theta_arr))/2
plt.plot(theta_arr/np.pi, gamma_arr)
plt.xlabel(r'$\theta_{ij} / \pi$')
plt.ylabel(r'$\gamma_{ij}$')
plt.title(r'$\gamma_{ij}(\theta_{ij}) = [1-\cos{(\theta_{ij})}]/2$')

In [ ]:
Gamma_arr = 3/2*gamma_arr *np.log(gamma_arr) - 1/4* gamma_arr + 1/2
plt.plot(theta_arr/np.pi, Gamma_arr)
plt.xlabel(r'$\theta_{ij} /\pi$')
plt.ylabel('$\Gamma_{ij}$')
plt.title(r'$\Gamma_{ij}(\theta_{ij}) = \frac{3}{2}\gamma_{ij} \ln (\gamma_{ij}) - \frac{1}{4}\gamma_{ij} + \frac{1}{2}$')

# Power Spectral Density
$S_h$, the one-sided power spectral density of the GW signal in the timing residuals
$$ S_h = \frac{h_c^2}{12 \pi ^2 f_k^3}$$

$S_{h0}$, the expected one-sided power spectral density of the GW signal
$$ S_{h0} = \frac{\mathcal{A}^2 \mathrm{yr}^{-4/3}}{12\pi^2} f^{-13/3} $$
where $\mathcal{A}$ is the fiducial characteristic strain amplitude such that 
$$h_c = \mathcal{A} [f/\mathrm{yr}^{-1}]^{-2/3}$$
Rosado et al. approximate as
$$ S_{h0} \approx S_h = \frac{h_c^2}{12 \pi ^2 f_k^3}$$

In [ ]:
def spectral_density(hc_bg, freqs):
    """ Calculate the spectral density S_h(f_k) ~ S_h0(f_k) at the kth frequency

    Parameters
    ----------
    hc_bg : (F,) 1D array of scalars
        Characteristic strain of the background at each frequency. 
    freqs : (F,) 1Darray of scalars
        Frequency bin centers corresponding to each strain

    Returns
    -------
    S_h : (F,) 1Darray of scalars
        Actual (S_h) or ~construction (S_h0) value of the background spectral density. 
        In units of [freqs]^-3
    """

    S_h = hc_bg**2 / (12 * np.pi**2 * freqs**3)
    return S_h

In [ ]:
print('fobs', fobs[8])
print('hc-bg', hc_bg[8,0])
Sh_bg = spectral_density(hc_bg[:,0], fobs) # spectral density of bg, using 0th realization
Sh0_bg = Sh_bg # approximation used in Rosado et al. 2015
print(Sh_bg)
print(Sh_bg[8])

# Noise spectral density $P_i$ 
$$ S_i = 2 \Delta t \sigma_i^2 + S_{h,\mathrm{rest}}$$
where $2 \Delta t \sigma_i ^2$ is the contribution from the pulsar's white noise and $S_{h,\mathrm{rest}}$ is from all the other SBHBs except for the max, in our single source detection. $P_i$ is just the $2 \Delta t \sigma_i^2$ part.

In [ ]:
def white_noise(delta_t, sigma_i):
    """ Calculate the white noise for a given pulsar 2 /Delta t sigma_i^2
    
    Parameters
    ----------
    delta_t : scalar
        Detection cadence, in seconds.
    sigma_i : scalar
        Error/stdev/variance? for the ith pulsar, in seconds.

    Returns
    -------
    P_i : scalar
        Noise spectral density for the ith pulsar, for bg detection.
        For single source detections, the noise spectral density S_i must also 
        include red noise from all but the loudest single sources, S_h,rest.

    """
    P_i = 2 * delta_t * sigma_i**2
    return P_i

In [ ]:
SIGMAS = np.linspace(1e-5, 1e-5,4) # (P,) 1Darray of scalars, sigma_i of each pulsar
print(cad5/YR, SIGMAS/YR, 'in years') #  cad and SIGMA are in seconds
print(cad5, SIGMAS, 'in seconds') #  cad and SIGMA are in seconds

noise_i = white_noise(cad5, SIGMAS) 
print('P_i =', noise_i, 's^3 =', noise_i/YR**3,' yrs^3')

### A-statistic Overview
When we maximize $\mu/\sigma_0 = \langle X \rangle / \sqrt{\mathrm{var}(X)_0}$

$$ \mu_1 = 2 \sum_k \sum_{ij} \frac{\Gamma_{ij}^2 S_h S_{h0}}{P_i P_j}$$

### B-statistic Overview

$$\mu_1 = 1\sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_h S_{h0}}{[P_i + S_{h0}] [P_j + S_{h0}] + \Gamma_{ij}^2 S_{h0}^2} $$

$$ \sigma_0^2 = 2\sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_{h0}^2 P_i P_j  }{\big[ [P_i + S_{h0}] [P_j +S_{h0}] + \Gamma_{ij}^2 S_{h0}^2  \big]^2  } $$

$$ \sigma_1^2 = 2 \sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_{h0}^2 \big[ [P_i + S_h] [P_j + S_h] + \Gamma_{ij}^2 S_h^2   \big]  }{\big[[P_i + S_{h0}][P_j + S_{h0}] + \Gamma_{ij}^2 S_{h0}^2  \big]^2  } $$


* $S_h(f_k)$ := actual value of the spectral density in the background
$$ S_h = \frac{h_c^2}{12 \pi ^2 f_k^3}$$
* $S_{h0}(f_k)$ := value of the spectral density used to construct the statistic
$$ S_{h0} \approx S_h$$
* $P_i$ = $S_i$ := noise spectral density
$$ S_i = 2 \Delta t \sigma_i^2 + S_{h,\mathrm{rest}}$$
* $2 \Delta t \sigma_i^2$ := the contribution from the pulsar's white noise
* $S_{h,\mathrm{rest}}$ := an additional red noise term produced by all other SBHBs at the same frequency bin
$$ S_{h,\mathrm{rest}} = \frac{h_{c,\mathrm{rest}}^2}{f} \frac{1}{12 \pi^2 f^2}$$


# mu_1
$$\mu_1 = 1\sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_h S_{h0}}{[P_i + S_{h0}] [P_j + S_{h0}] + \Gamma_{ij}^2 S_{h0}^2} $$


In [ ]:
def mean1_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg):
    """ Calculate mu_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    mu_1B : 
        Expected value for the B statistic

    Follows Eq. (A16) from Rosado et al. 2015.
    """
    mu_1B = 0
    for ii in range(len(noise_i)):
        P_i = noise_i[ii]
        for jj in range(len(noise_i)): 
            if(jj>ii):
                P_j = noise_i[jj]
                Gamma = Gamma_ij[ii,jj]
                for kk in range(len(Sh_bg)):
                    Sh = Sh_bg[kk]
                    Sh0 = Sh0_bg[kk]
                    mu_1B += ((Gamma**2 * Sh * Sh0)
                            /((P_i+Sh0) * (P_j+Sh0) 
                                + Gamma**2 * Sh0**2))
    mu_1B *= 2
    return mu_1B

def mean1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg):
    """ Calculate mu_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    mu_1B : 
        Expected value for the B statistic

    Follows Eq. (A16) from Rosado et al. 2015.
    """
    
    # to get sum term in shape (P,P,F) for ii,jj,kk we want:
    # Gamma_ij in shape (P,P,1)
    # Sh0 and Sh in shape (1,1,F)
    # P_i in shape (P,1,1)
    # P_j in shape (1,P,1)

    numer = (Gamma_ij[:,:,np.newaxis] **2 
            * Sh_bg[np.newaxis, np.newaxis, :]
            * Sh0_bg[np.newaxis, np.newaxis, :])
    denom = ((noise_i[:, np.newaxis, np.newaxis] + Sh0_bg[np.newaxis,np.newaxis,:])
               * (noise_i[np.newaxis, :, np.newaxis] + Sh0_bg[np.newaxis,np.newaxis,:])
               + Gamma_ij[:,:,np.newaxis]**2 * Sh0_bg[np.newaxis, np.newaxis, :]**2)
    
    # NOTE: still uses for loops for the part, might be worth changing
    sum =0
    for ii in range(len(numer)):
        for jj in range(len(numer[0])):
            if(jj>ii):
                sum += np.sum(numer[ii,jj,:]/denom[ii,jj,:])
    mu_1B = 2*sum
    return mu_1B


In [ ]:
mu_1B_loops = mean1_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
mu_1B_ndars = mean1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
print(mu_1B_loops, mu_1B_ndars)

# sigma_0
$$ \sigma_0^2 = 2\sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_{h0}^2 P_i P_j  }{\big[ [P_i + S_{h0}] [P_j +S_{h0}] + \Gamma_{ij}^2 S_{h0}^2  \big]^2  } $$



In [ ]:
def sigma0_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg):
    """ Calculate sigma_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    sigma_0B : Scalar
        
    
    Follows Eq. (A17) from Rosado et al. 2015.
    """

    sigma_0B = 0 # sigma_1 squared
    for ii in range(len(noise_i)):
        P_i = noise_i[ii]
        for jj in range(len(noise_i)): 
            if(jj>ii):
                P_j = noise_i[jj]
                Gamma = Gamma_ij[ii,jj]
                for kk in range(len(Sh_bg)):
                    Sh0 = Sh0_bg[kk]
                    numer = Gamma**2 * Sh0**2 * P_i * P_j
                    denom = ((P_i + Sh0) * (P_j + Sh0) 
                             + Gamma**2 * Sh0**2)**2
                    sigma_0B += (numer/denom)
    
    sigma_0B = np.sqrt(2*sigma_0B)
    return sigma_0B


def sigma0_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg):
    """ Calculate sigma_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    sigma_0B : Scalar
        

    Follows Eq. (A17) from Rosado et al. 2015.
    """

    # to get sum term in shape (P,P,F) for ii,jj,kk we want:
    # Gamma_ij in shape (P,P,1)
    # Sh0 and Sh in shape (1,1,F)
    # P_i in shape (P,1,1)
    # P_j in shape (1,P,1)

    numer = (Gamma_ij[:,:,np.newaxis]**2 * Sh0_bg[np.newaxis,np.newaxis,:]**2 
             * noise_i[:,np.newaxis,np.newaxis] * noise_i[np.newaxis,:,np.newaxis])
    denom = ((noise_i[:,np.newaxis,np.newaxis] + Sh0_bg[np.newaxis, np.newaxis,:])
              * (noise_i[np.newaxis,:,np.newaxis] + Sh0_bg[np.newaxis,np.newaxis,:])
             + Gamma_ij[:,:,np.newaxis]**2 * Sh0_bg[np.newaxis,np.newaxis,:]**2)**2
    
    # NOTE: still uses for loops for the part, might be worth changing
    sum =0
    for ii in range(len(numer)):
        for jj in range(len(numer[0])):
            if(jj>ii):
                sum += np.sum(numer[ii,jj,:]/denom[ii,jj,:])
    sigma_0B = np.sqrt(2*sum)
    return sigma_0B



In [ ]:
sigma_0B_loops = sigma0_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
sigma_0B_ndars = sigma0_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
print(sigma_0B_loops, sigma_0B_ndars)

# sigma_1
$$ \sigma_1^2 = 2 \sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_{h0}^2 \big[ [P_i + S_h] [P_j + S_h] + \Gamma_{ij}^2 S_h^2   \big]  }{\big[[P_i + S_{h0}][P_j + S_{h0}] + \Gamma_{ij}^2 S_{h0}^2  \big]^2  } $$

In [ ]:
def sigma1_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg):
    """ Calculate sigma_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    sigma_1B : Scalar
        

    Follows Eq. (A18) from Rosado et al. 2015.
    """

    sigma_1B = 0 # sigma_1 squared
    for ii in range(len(noise_i)):
        P_i = noise_i[ii]
        for jj in range(len(noise_i)): 
            if(jj>ii):
                P_j = noise_i[jj]
                Gamma = Gamma_ij[ii,jj]
                for kk in range(len(Sh_bg)):
                    Sh = Sh_bg[kk]
                    Sh0 = Sh0_bg[kk]
                    numer = (Gamma**2 * Sh0**2 * 
                             ((P_i + Sh) * (P_j + Sh) 
                              + Gamma**2 * Sh**2))
                    denom = ((P_i + Sh0) * (P_j + Sh0) 
                             + Gamma**2 * Sh0**2)**2
                    sigma_1B += (numer/denom)
    
    sigma_1B = np.sqrt(2*sigma_1B)
    return sigma_1B


def sigma1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg):
    """ Calculate sigma_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    sigma_1B : Scalar
        

    Follows Eq. (A18) from Rosado et al. 2015.
    """

    # to get sum term in shape (P,P,F) for ii,jj,kk we want:
    # Gamma_ij in shape (P,P,1)
    # Sh0 and Sh in shape (1,1,F)
    # P_i in shape (P,1,1)
    # P_j in shape (1,P,1)

    numer = (Gamma_ij[:,:,np.newaxis]**2 * Sh0_bg[np.newaxis,np.newaxis,:]**2 
             * ((noise_i[:,np.newaxis,np.newaxis] + Sh_bg[np.newaxis,np.newaxis,:])
                * (noise_i[np.newaxis,:,np.newaxis] + Sh_bg[np.newaxis,np.newaxis,:])
                + Gamma_ij[:,:,np.newaxis]**2 * Sh_bg[np.newaxis,np.newaxis,:]**2))
             
    denom = ((noise_i[:,np.newaxis,np.newaxis] + Sh0_bg[np.newaxis, np.newaxis,:])
              * (noise_i[np.newaxis,:,np.newaxis] + Sh0_bg[np.newaxis,np.newaxis,:])
             + Gamma_ij[:,:,np.newaxis]**2 * Sh0_bg[np.newaxis,np.newaxis,:]**2)**2
    
    # NOTE: still uses for loops for the part, might be worth changing
    sum =0
    for ii in range(len(numer)):
        for jj in range(len(numer[0])):
            if(jj>ii):
                sum += np.sum(numer[ii,jj,:]/denom[ii,jj,:])
    sigma_1B = np.sqrt(2*sum)
    return sigma_1B



In [ ]:
sigma_1B_loops = sigma1_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
sigma_1B_ndars = sigma1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
print(sigma_1B_loops, sigma_1B_ndars)

## check using S/N_B
$$S/N_B = \Bigg[ 2 \sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_h^2}{P_iP_j + S_h[P_i + P_j] + S_h^2[1+\Gamma_{ij}^2]}   \Bigg]^{1/2} $$
should equal 
$$ S/N_B \equiv \frac{\mu_1}{ \sigma_1} $$

# Background Detection Probability
$$ \gamma_{bg} = \frac{1}{2} \mathrm{erfc} \big[ \frac{\sqrt{2} \sigma_0 \mathrm{erfc}^{-1}(2\alpha_0) - \mu_1}{\sqrt{2} \sigma_1}\big]

In [ ]:
ALPHA0 = 0.001 # false alarm probability (FAP)
GAMMA0 = 0.95 # detection probability

def bg_detection_probability(sigma_0, sigma_1, mu_1, alpha_0):
    """ Calculate the background detection probability, gamma_bg.

    Parameters
    ----------
    sigma_0 : scalar
        Standard deviation of stochastic noise processes.
    sigma_1 : scalar
        Standard deviation of GWB PDF.
    mu_1 : scalar
        Mean of GWB PDF.
    alpha_0 : scalar
        False alarm probability max.

    Returns
    -------
    dp_bg : scalar
        Background detection probability.

        
    Follows Rosado et al. 2015 Eq. (15)
    """
    temp = ((np.sqrt(2) * sigma_0 * sp.special.erfcinv(2*alpha_0) - mu_1)
            /(np.sqrt(2) * sigma_1))
    dp_bg = .5 * sp.special.erfc(temp)
    return dp_bg

In [ ]:
print('fobs (s^-1):', holo.utils.stats(fobs))
print('hc_bg:', holo.utils.stats(hc_bg[:,0]))
print('cadence (s)', cad5)
print('pulsar thetas (pi rad):', THETAS/np.pi)
print('pulsar sigmas (s):', SIGMAS)
print('P_i (s^3):', holo.utils.stats(noise_i))
print('S_h (s^3):', holo.utils.stats(Sh_bg))
print('sigma_0:',sigma_0B_loops)
print('sigma_1:',sigma_1B_loops)
print('mu_1:', mu_1B_loops)
gamma_bg = bg_detection_probability(sigma_0B_loops, sigma_1B_loops,
                                    mu_1B_loops, ALPHA0)
print(gamma_bg)

In [ ]:
def SNR_B(noise_i, Gamma_ij, Sh_bg):
    """ Calculate S/N_B for the background, using P_i, Gamma_ij, S_h and S_h0
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    Gamma_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    SNR_B : Scalar
        Signal to noise ratio assuming the B statistic
        

    Follows Eq. (A19) from Rosado et al. 2015.
    """


    # to get sum term in shape (P,P,F) for ii,jj,kk we want:
    # Gamma_ij in shape (P,P,1)
    # Sh0 and Sh in shape (1,1,F)
    # P_i in shape (P,1,1)
    # P_j in shape (1,P,1)

    numer = Gamma_ij[:,:,np.newaxis]**2 * Sh_bg[np.newaxis,np.newaxis,:]**2
    denom = (noise_i[:,np.newaxis,np.newaxis] * noise_i[np.newaxis,:,np.newaxis]
             + Sh_bg[np.newaxis,np.newaxis,:] * (noise_i[:,np.newaxis,np.newaxis]+noise_i[np.newaxis,:,np.newaxis])
             + Sh_bg[np.newaxis,np.newaxis,:]**2 * (1 + Gamma_ij[:,:,np.newaxis]**2))

    sum =0
    for ii in range(len(numer)):
        for jj in range(len(numer[0])):
            if(jj>ii):
                sum += np.sum(numer[ii,jj,:]/denom[ii,jj,:])
    SNR_B = np.sqrt(2*sum)
    return SNR_B

    

Timeit

In [ ]:
%timeit mu_1B_loops = mean1_Bstatistic_loops(noise_i, Gamma_ij, Sh_bg, Sh0_bg)
%timeit mu_1B_ndars = mean1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)

# All together, detect_bg

In [ ]:
# example 5
edges, number, fobs, exname = ss.example5(print_test=False)
hc_bg, hc_ss, ssidx, hsamp, bgpar, sspar = ss.ss_by_cdefs(edges, number, 30, params=True)
dur = 10.0*YR
cad = .2*YR

In [ ]:
def detect_bg(THETAS, SIGMAS, fobs, cad, hc_bg, return_all = False):
    """ Calculate the background detection probability, and all intermediary steps.

    Parameters
    ----------
    THETAS : (P,) 1Darray of scalars
        Angular position of each pulsar in radians.
    SIGMAS : (P,) 1Darray of scalars
        Sigma_i of each pulsar in seconds.
    fobs : (F,) 1Darray of scalars
        Frequency bin centers in hertz.
    cad : scalar
        Cadence of observations in seconds.
    hc_bg : (F,)
        Characteristic strain of the background at each frequency.
    return_all : Bool
        Whether to return all parameters or just dp_bg

    Returns
    -------
    dp_bg : scalar
        Background detection probability
    Gamma_ij : (P, P) 2D Array
        Overlap reduction function.
        Only returned if return_all = True.
    Sh_bg : (F,) 1Darray
        Spectral density
        Only returned if return_all = True.
    noise_i : (P,) 1Darray
        Spectral noise density of each pulsar.
        Only returned if return_all = True.
    mu_1B : scalar
        Expected value for the B statistic.
        Only returned if return_all = True.
    sigma_0B : scalar
    sigma_1B : scalar

    """
    # Overlap Reduction Function
    num = len(THETAS) # number of pulsars, P
    Gamma_ij = np.zeros((num, num)) # (P,P) 2Darray of scalars, Overlap reduction function between all puolsar
    for ii in range(num):
        for jj in range(num):
            Gamma_ij[ii,jj] = overlap_reduction_function(THETAS[ii], THETAS[jj], ii, jj)

    # Spectral Density
    Sh_bg = spectral_density(hc_bg[:], fobs) # spectral density of bg, using 0th realization
    Sh0_bg = Sh_bg # approximation used in Rosado et al. 2015

    # Noise 
    noise_i = white_noise(cad, SIGMAS) 

    mu_1B = mean1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)

    sigma_0B = sigma0_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)

    sigma_1B = sigma1_Bstatistic_ndars(noise_i, Gamma_ij, Sh_bg, Sh0_bg)

    dp_bg = bg_detection_probability(sigma_0B, sigma_1B, mu_1B, ALPHA0)

    if(return_all):
        return dp_bg, Gamma_ij, Sh_bg, noise_i, mu_1B, sigma_0B, sigma_1B
    else:
        return dp_bg

In [ ]:
THETAS = np.array([0, np.pi/6, np.pi, 7/6*np.pi]) # (P,) 1Darray of scalars, angular sky position of each pulsar
SIGMAS = np.linspace(1e-5, 1e-5,4) # (P,) 1Darray of scalars, sigma_i of each pulsar
ALPHA0 = 0.001
dp_bg, Gamma_ij, Sh_bg, noise_i, mu_1B, sigma_0B, sigma_1B = \
    detect_bg(THETAS, SIGMAS, fobs, cad, hc_bg[:,1], return_all=True)

print('fobs     \t%s nHz:' % str(holo.utils.stats(fobs)))
print('hc_bg:   \t%s' % str(holo.utils.stats(hc_bg[:,0])))
print('cadence: \t%.2e s' % cad)
print('pulsar thetas (pi rad):', THETAS/np.pi)
print('pulsar sigmas (s):', SIGMAS)
print('P_i (s^3):\t%s' % str(holo.utils.stats(noise_i)))
print('S_h (s^3):\t%s' % str(holo.utils.stats(Sh_bg)))
print('sigma_0: \t%.2e' % sigma_0B)
print('sigma_1: \t%.2e' % sigma_1B)
print('mu_1:    \t%.2e' % mu_1B)
print('DP_bg:   \t%.2e' % dp_bg)

Print Info

In [ ]:
num_pulsars = 40
THETAS = np.linspace(.00001, 2*np.pi, num_pulsars) # (P,) 1Darray of scalars, angular sky position of each pulsar
SIGMAS = np.linspace(1e-6, 2e-6, num_pulsars) # (P,) 1Darray of scalars, sigma_i of each pulsar
ALPHA0 = 0.001
dp_bg, Gamma_ij, Sh_bg, noise_i, mu_1B, sigma_0B, sigma_1B = \
    detect_bg(THETAS, SIGMAS, fobs, cad, hc_bg[:,1], return_all=True)
print('Number of pulsars:',num_pulsars)
print('\nOVERLAP')
print('thetas (pi):\t%s' % str(holo.utils.stats(THETAS/np.pi)))
print('Gamma_ij:\t%s' % str(holo.utils.stats(Gamma_ij)))
# print('Gamma_ij:\t', Gamma_ij)

print('\nNOISE')
print('cadence (s): \t%.2e' % cad)
print('sigmas (s): %s', str(holo.utils.stats(SIGMAS)))
print('P_i (s^3):\t%s' % str(holo.utils.stats(noise_i)))

print('\nSIGNAL')
print('fobs (nHz): \t%s' % str(holo.utils.stats(fobs*10**9)))
print('hc_bg:   \t%s' % str(holo.utils.stats(hc_bg[:,0])))
print('S_h (s^3):\t%s' % str(holo.utils.stats(Sh_bg)))

print('\nB STATISTICS')
print('sigma_0: \t%.2e' % sigma_0B)
print('sigma_1: \t%.2e' % sigma_1B)
print('mu_1:    \t%.2e' % mu_1B)

print('\nDETECTION PROB')
print('DP_bg:   \t%.2e' % dp_bg)
print('SNR_B:   \t%.2f' % (mu_1B/sigma_1B))

In [ ]:
SNR = SNR_B(noise_i, Gamma_ij, Sh_bg)
print(SNR)
# Check! It matches

# Single Sources
Absent of a signal: $$p_0 (\mathcal{F}_e) = \mathcal{F_e}e^{-\mathcal{F_e}}$$

With a signal: $$p_1(\mathcal{F}_e, \rho) = \frac{[2\mathcal{F}_e]^{1/2}}{\rho} I_1 (\rho \sqrt{2\mathcal{F}_e}) e^{-\mathcal{F}_e-1/2\rho^2} $$

### DP for a specific $f$ bin
Probability of detecting one binary in a particular frequency bin
$$ \gamma_i = \int_{\overline{\mathcal{F}}_e}^\infty p_1 (\mathcal{F}_e, \rho) d\mathcal{F}_e = \int_{\overline{\mathcal{F}}_e}^\infty \frac{[2\mathcal{F}_e]^{1/2}}{\rho}I_1(\rho \sqrt{2 \mathcal{F}_e})e^{-\mathcal{F}_e -\frac{1}{2}\rho^2}d\mathcal{F}_e $$


### DP any $f$ bin
To detect *any* single source, need probability of detecting one in any frequency bin, given by:
$$ \gamma_S = 1 - \prod_i[1-\gamma_i]

Some variables
* characterstic frequency of loudest binary, $h_c^\mathrm{max} in each frequency bin.
* Optimal signal to noise ratio for a SBHB, $\mathrm{S/N_S}$ 
$$ \mathrm{S/N_S} = \Big[ \sum_{i=1}^M \mathrm{S/N}_i^2 \Big]^{1/2}$$
* Signal to noise ratio of the signal in an individual pulsar $\mathrm{S/N}_i$. This a function of amplitude $A$ (function of $\mathcal{M}$, $f$, $z$, and $r$), frequency $f$, GW phase $\Phi = 2\pi f t$, and antenna pattern functions $F_i^+$ and $F_i^\times$. Assume monochromatic because observation time scale is so much shorter than SBHB evolution timescale, can solve $\mathrm{S/N}_i^2$ integral analytically to get it as a function of $A$, $a$, $f$, $F_i^+$, $F_i^\times$, $\Phi_T$, $\Phi_0$.

## Unitary Vectors
First, unitary vectors as functions of $\phi$, $\theta$, and $\xi$. I think these are belonging to the source, but I don't understand what the $\xi$ is.

$$\hat{m} = +[\sin (\phi)\cos(\xi) - \sin (\xi) cos(\phi) cos(\theta)] \hat{x} 
\\
- [\cos (\phi) \cos(\xi) + \sin(\xi) \sin(\phi) \cos(\theta)] \hat{y} 
\\
+ [\sin)(\xi)\sin(\phi)]\hat{z}$$

$$ \hat{n} = +[-\sin(\phi) \sin(\xi) - \cos(\xi) \cos(\phi) \cos(\theta)] \hat{x} \\
+[\cos(\phi) \sin(\xi) - \cos(\xi) \sin(\phi) \cos(\theta)] \hat{y} \\
+[\cos(\xi) \sin(\theta)]\hat{z} $$

$$\hat{\Omega} = -\sin(\theta) \cos(\phi) \hat{x}  - \sin(\theta) \sin(\phi) \hat{y} - \cos(\theta) \hat{z} $$

$$ \hat{p}_i = \sin(\theta_i) \cos(\phi_i) \hat{x} + \sin(\theta_i) \sin(\phi_i) \hat{y} + \cos(\theta_i) \hat{z} $$

In [ ]:
def m_unitary_vector(phi, theta, xi):
    """ Calculate the unitary vector m-hat for the antenna pattern functions.
    
    Parameters
    ----------
    phi : scalar
    theta : scalar
    xi : scalar
    
    
    Returns
    -------
    m_hat : (3,) vector 
        Unitary vector m-hat with x, y, and z components at 
        index 0, 1, and 2, respectively.
        
    """
    mhat_x = (np.sin(phi) * np.cos(xi) 
              - np.sin(xi) * np.cos(phi) * np.cos(theta))
    mhat_y = -(np.cos(phi) * np.cos(xi)
               + np.sin(xi) * np.sin(phi) * np.cos(theta))
    mhat_z = (np.sin(xi) * np.sin(theta))

    m_hat = np.array([mhat_x, mhat_y, mhat_z])
    return m_hat

def n_unitary_vector(phi, theta, xi):
    """ Calculate the unitary vector n-hat for the antenna pattern functions.
    
    Paramters
    ---------
    phi : scalar
    theta : scalar
    xi : scalar
    
    Returns
    -------
    n_hat : (3,) vector
        Unitary vector n-hat.
        
    """

    nhat_x = (- np.sin(phi) * np.sin(xi) 
              - np.cos(xi) * np.cos(phi) * np.cos(theta))
    nhat_y = (np.cos(phi) * np.sin(xi) 
              - np.cos(xi) * np.sin(phi) * np.cos(theta))
    nhat_z = np.cos(xi) * np.sin(theta)

    n_hat = np.array([nhat_x, nhat_y, nhat_z])
    return n_hat

def Omega_unitary_vector(phi, theta):
    """ Calculate the unitary vector n-hat for the antenna pattern functions.
    
    Paramters
    ---------
    phi : scalar
    theta : scalar
    
    Returns
    -------
    Omega_hat : (3,) vector
        Unitary vector Omega-hat.
    """

    Omegahat_x = - np.sin(theta) * np.cos(phi)
    Omegahat_y = - np.sin(theta) * np.sin(phi)
    Omegahat_z = - np.cos(theta)

    Omega_hat = np.array([Omegahat_x, Omegahat_y, Omegahat_z])
    return Omega_hat

def pi_unitary_vectory(phi_i, theta_i):
    """ Calculate the unitary vector p_i-hat for the ith pulsar.
    
    Parameters
    ----------
    phi_i : scalar
    theta_i : scalar
    
    Returns
    -------
    pi_hat : (3,) vector
    
    """

    pihat_x = np.sin(theta_i) * np.cos(phi_i)
    pihat_y = np.sin(theta_i) * np.sin(phi_i)
    pihat_z = np.cos(theta_i)

    pi_hat = np.array([pihat_x, pihat_y, pihat_z])
    return pi_hat


## Antenna Pattern Functions
$$ F_i^+ = \frac{1}{2}  \frac{[\hat{m}\cdot \hat{p}_i]^2 - \hat{n} \cdot \hat{p}_i]^2}{1  + \hat{\Omega} \cdot \hat{p}_i} $$
 
 $$ F_i^\times = \frac{\hat{m} \cdot \hat{p}_i] [\hat{n} \cdot \hat{p}_i]}{1 + \hat{\Omega} \cdot \hat{p}_i}

In [ ]:
def antenna_pattern_functions(m_hat, n_hat, Omega_hat, pi_hat):
    """ + antenna pattern function for the ith pulsar.
    
    Parameters
    ----------
    m_hat : (3,) vector
    n_hat : (3,) vector
    Omega_hat : (3,) vector
    pi_hat : (3,) vector
        for the ith pulsar.
        
    Returns
    -------
    Fi_plus : scalar
    Fi_cross : scalar
    
    """
    
    denom = 1 + np.dot(Omega_hat, pi_hat)
    Fi_plus = ((np.dot(m_hat, pi_hat)**2 - np.dot(n_hat, pi_hat)**2) 
               / denom / 2)
    Fi_cross = np.dot(m_hat, pi_hat) * np.dot(n_hat, pi_hat) / denom
    
    return Fi_plus, Fi_cross
    

## Amplitude, A
$$ h_s = \frac{8}{10^{1/2}} \frac{(G\mathcal{M})^{5/3}}{c^4 d_L} (2\pi f_r)^{2/3}  = \frac{2^{2/3}\times8}{2\times10^{1/2}} \times \big[ 2 \frac{(G\mathcal{M})^{5/3}}{c^4 d_L} (\pi f_r)^{2/3} \big] =  \frac{4(2)^{1/6}}{\sqrt{5}} A $$
$$ h_{c,ss}^2 = h_s^2 / dlnf = h_s^2 * f / df $$
$$ h_{c,ss} = h_s \sqrt{f/df} \frac{4(2)^{1/6}}{\sqrt{5}} A * f / df$$
$$ A = \frac{\sqrt{5}}{4 (2)^{1/6}} \sqrt{\frac{df}{f}} h_{c,ss} $$

In [ ]:
def amplitude(hc_ss, f, df):
    """ Calculate the amplitude from the single source to use in DP calculations
    
    Parameters
    ----------
    hc_ss : scalar, ndarray
        Characteristic strain of a single source.
    f : scalar, ndarray
        Frequency.
    df : scalar, ndarray
        Frequency bin width.

    Returns
    -------
    Amp : scalar, ndarray
        Dimensionless amplitude, A, of each single source.
    
    """

    Amp = hc_ss * np.sqrt(5) / 4 / 2**(1/6) *np.sqrt(df/f)
    return Amp

## Polarization contributions
$$ a = 1 + \cos^2 \iota $$
$$ b = -2 \cos \iota $$

What is iota?

## SNR for a single pulsar 
$$ S/N_i^2 = \frac{A^2}{S_i 8 \pi^3 f^3} \bigg[a^2[F_i^+]^2 \big[\Phi_T [1 + 2 \sin^2(\Phi_0)]
+ \cos(\Phi_T)[-\sin(\Phi_T) + 4 \sin(\Phi_0)] - 4\sin(\Phi_0)\big] \\
+ b^2[F_i^\times]^2 \big[\Phi_T[1+2\cos^2(\Phi_0)] + \sin(\Phi_T)[\cos(\Phi_T) - 4\cos(\Phi_0)]\big] \\
- 2ab F_i^+ F_i^\times \big[2\Phi_T \sin(\Phi_0) \cos(\Phi_0) \\
+ \sin(\Phi_T)[\sin(\Phi_T) - 2\sin(\Phi_0) + 2\cos(\Phi_T) \cos(\Phi_0) - 2\cos(\Phi_0)]\big]\bigg] $$

$$ A = 2 \frac{G^{5/3} \mathcal{M}^{5/3} [\pi f [1+z]]^{2/3}}{c^4 r} \quad (5)$$
$$ h = A\sqrt{\frac{1}{2}[a^2+b^2]} \quad (4)$$

In [ ]:
def SNR_pulsar(A, Fi_plus, Fi_cross, iota, Phi_T, Phi_0, S_i)
    """ 
    A : scalar, array
        Dimensionless strain amplitude.
    
    """

    a = 1 + (np.cos(iota))**2
    b = -2 * np.cos(iota)
    Phi_T = 2 * 

## Noise Spectral Density (for SS)
Noise from all other sources in the same frequency bin except for the loudest
$$ S_{h,\mathrm{rest}} = \frac{h_{c,\mathrm{rest}}^2}{f} \frac{1}{12 \pi^2 f^2} $$
Included in total noise spectral density
$$S_i = 2\Delta t \sigma_i^2 + S_{h,\mathrm{rest}}$$

In [ ]:
def pulsar_noise():
    """ Calculate the noise spectral density of the ith pulsar

    Parameters
    ----------

    Returns
    -------

    """
    P_i = white_noise(delta_t, sigma_i)
    Sh_rest = 0
    for ii in range(len(pulsars)):
        S_i = P_i + Sh_rest